In [1]:
import geopandas as gpd

# Path to your file
file_path = "/home/rishabh.mondal/Brick-Kilns-project/ijcai_2025_kilns/after_hand_validation_data/india_v5/final/punjab.geojson"

# Read the geojson
gdf = gpd.read_file(file_path)

# Inspect the data
print(gdf.head())
print(gdf.crs)
print(gdf.shape)


  class_name  confidence    max_lon    min_lon    max_lat    min_lat  \
0       FCBK    0.961565  75.502699  75.500911  30.024692  30.023437   
1       FCBK    0.283720  75.166663  75.165398  30.304030  30.303121   
2     Zigzag    0.942558  76.541805  76.540588  30.423564  30.422053   
3       FCBK    0.833221  76.323847  76.322288  30.130022  30.129444   
4     Zigzag    0.900114  75.921401  75.920717  30.784777  30.783460   

   center_lat  center_lon  width_of_box  height_of_box  ...  resolution  \
0   30.024064   75.501805    172.456283     139.068523  ...        None   
1   30.303576   75.166030    121.758011     100.708351  ...        None   
2   30.422809   76.541197    116.902170     167.502987  ...        None   
3   30.129733   76.323067    150.255189      64.060735  ...        None   
4   30.784118   75.921059     65.548579     145.982224  ...        None   

                       source  task_name geo_box  style  type   x   y  \
0  Drawn|Azure Maps Satellite              

In [2]:
import geopandas as gpd

src = "/home/rishabh.mondal/Brick-Kilns-project/ijcai_2025_kilns/after_hand_validation_data/india_v5/final/punjab.geojson"
dst = "punjab.kml"

gdf = gpd.read_file(src)

# KML expects WGS84
if gdf.crs is None or gdf.crs.to_epsg() != 4326:
    gdf = gdf.to_crs(epsg=4326)

# write a single layer to KML
gdf.to_file(dst, driver="KML")
print("Wrote", dst)


/opt/anaconda3/envs/rishabh_sat/lib/python3.12/site-packages/pyogrio/raw.py:709: RuntimeWarning: Value 'b11924e38bb578e56279f90549e0901ebf5fdcf844ea2bafd9b8fe069e2e02b2' of field punjab.width_of_object parsed incompletely to real 0.
  ogr_write(
/opt/anaconda3/envs/rishabh_sat/lib/python3.12/site-packages/pyogrio/raw.py:709: RuntimeWarning: Value 'Drawn|Azure Maps Satellite' of field punjab.zoom parsed incompletely to real 0.
  ogr_write(
/opt/anaconda3/envs/rishabh_sat/lib/python3.12/site-packages/pyogrio/raw.py:709: RuntimeWarning: Value 'c78579ccd4fdf827a691ca2e4610c4d2b1d1d26d63259e5bcecea22a1a39b85c' of field punjab.width_of_object parsed incompletely to real 0.
  ogr_write(
/opt/anaconda3/envs/rishabh_sat/lib/python3.12/site-packages/pyogrio/raw.py:709: RuntimeWarning: Value 'e85395b2b9e0a061535e3780b499fbec0ac294cd7d66c5560e52e7924ff868db' of field punjab.width_of_object parsed incompletely to real 0.
  ogr_write(
/opt/anaconda3/envs/rishabh_sat/lib/python3.12/site-packages/pyog

Wrote punjab.kml


/opt/anaconda3/envs/rishabh_sat/lib/python3.12/site-packages/pyogrio/raw.py:709: RuntimeWarning: Value '179f005f2ca960cb02c2fc679e7bd70bae26802c49681682d133b773f89e492a' of field punjab.width_of_object parsed incompletely to real 179.
  ogr_write(
/opt/anaconda3/envs/rishabh_sat/lib/python3.12/site-packages/pyogrio/raw.py:709: RuntimeWarning: Value '8bc3196e356305b3124b88f4810bd7c3c49c0a718fb97a1fbe1a368138059183' of field punjab.width_of_object parsed incompletely to real 8.
  ogr_write(
/opt/anaconda3/envs/rishabh_sat/lib/python3.12/site-packages/pyogrio/raw.py:709: RuntimeWarning: Value '[1.       0.283893 0.814539 0.283938 0.798909 0.244653 0.798798 0.244608
 0.814427 0.343586]' of field punjab.x parsed incompletely to real 0.
  ogr_write(
/opt/anaconda3/envs/rishabh_sat/lib/python3.12/site-packages/pyogrio/raw.py:709: RuntimeWarning: Value '0d110dafbd36043c2bd59074120c47f42c69e3270a2b2bf17969bc4243d87315' of field punjab.width_of_object parsed incompletely to real 0.
  ogr_write(


In [9]:
# Requirements: geopandas, simplekml, shapely
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point, LineString, Polygon, MultiPolygon, MultiLineString, MultiPoint, GeometryCollection
import simplekml
from collections import defaultdict
from pathlib import Path

# I/O
SRC = "/home/rishabh.mondal/Brick-Kilns-project/ijcai_2025_kilns/after_hand_validation_data/india_v5/final/punjab.geojson"
DST = "punjab_selected.kml"

# Load
gdf = gpd.read_file(SRC)

# Ensure WGS84 for KML
if gdf.crs is None or gdf.crs.to_epsg() != 4326:
    gdf = gdf.to_crs(epsg=4326)

# Columns: class_name, center_lat, center_lon, geometry
# Compute centers if missing
if "center_lat" not in gdf.columns or "center_lon" not in gdf.columns:
    centers = gdf.geometry.centroid
    gdf["center_lat"] = centers.y
    gdf["center_lon"] = centers.x

# If class column uses a different name, normalize here
# Try common variants; fall back to existing 'class_name'
for cand in ["class_name", "class", "cls", "label", "category"]:
    if cand in gdf.columns:
        gdf["class_name"] = gdf[cand]
        break
if "class_name" not in gdf.columns:
    raise ValueError("No class field found. Add/rename your class column to 'class_name'.")

# Keep only requested fields
gdf = gdf[["class_name", "center_lat", "center_lon", "geometry"]].copy()

# Add serial id 1..N
gdf.insert(0, "id", range(1, len(gdf) + 1))

# Color map per class (opaque; KML uses aabbggrr). Using simplekml named colors.
palette = [
    simplekml.Color.red,
    simplekml.Color.blue,
    simplekml.Color.green,
    simplekml.Color.yellow,
    simplekml.Color.purple,
    simplekml.Color.orange,
    simplekml.Color.cyan,
    simplekml.Color.pink,
    simplekml.Color.white,
    simplekml.Color.brown,
]
class_to_color = {}
next_color_idx = 0
for cname in gdf["class_name"].astype(str).unique():
    class_to_color[cname] = palette[next_color_idx % len(palette)]
    next_color_idx += 1

# KML build
kml = simplekml.Kml()
folders = defaultdict(lambda: None)

def ensure_folder(cname: str):
    if folders[cname] is None:
        folders[cname] = kml.newfolder(name=f"class={cname}")
    return folders[cname]

def style_for_class(feat, cname: str):
    col = class_to_color[cname]
    # Lines/polygons
    if hasattr(feat, "linestyle"):
        feat.linestyle.width = 2
        feat.linestyle.color = col
    if hasattr(feat, "polystyle"):
        feat.polystyle.color = simplekml.Color.changealphaint(80, col)  # semi-transparent fill
        feat.polystyle.outline = 1
    # Points
    if hasattr(feat, "iconstyle"):
        feat.iconstyle.color = col
        feat.iconstyle.scale = 1.1

def add_geom(geom, name: str, desc: str, cname: str, folder):
    if geom is None or geom.is_empty:
        return
    col = class_to_color[cname]

    if isinstance(geom, Point):
        p = folder.newpoint(name=name, coords=[(geom.x, geom.y)])
        p.description = desc
        style_for_class(p, cname)

    elif isinstance(geom, MultiPoint):
        for i, g in enumerate(geom.geoms):
            p = folder.newpoint(name=f"{name}_pt{i}", coords=[(g.x, g.y)])
            p.description = desc
            style_for_class(p, cname)

    elif isinstance(geom, LineString):
        ls = folder.newlinestring(name=name, coords=list(geom.coords))
        ls.description = desc
        style_for_class(ls, cname)

    elif isinstance(geom, MultiLineString):
        for i, g in enumerate(geom.geoms):
            ls = folder.newlinestring(name=f"{name}_ln{i}", coords=list(g.coords))
            ls.description = desc
            style_for_class(ls, cname)

    elif isinstance(geom, MultiPolygon):
        for i, poly in enumerate(geom.geoms):
            outer = list(poly.exterior.coords)
            inners = [list(r.coords) for r in poly.interiors]
            pg = folder.newpolygon(
                name=f"{name}_poly{i}",
                outerboundaryis=outer,
                innerboundaryis=inners
            )
            pg.description = desc
            style_for_class(pg, cname)

            # Add label at centroid of each polygon
            c = poly.centroid
            lbl = folder.newpoint(name=str(fid), coords=[(c.x, c.y)])
            lbl.style.labelstyle.scale = 1.2
            lbl.style.labelstyle.color = class_to_color[cname]



    elif isinstance(geom, MultiPolygon):
        for i, poly in enumerate(geom.geoms):
            outer = list(poly.exterior.coords)
            inners = [list(r.coords) for r in poly.interiors]
            pg = folder.newpolygon(name=f"{name}_poly{i}", outerboundaryis=outer, innerboundaryis=inners)
            pg.description = desc
            style_for_class(pg, cname)

    elif isinstance(geom, GeometryCollection):
        for i, g in enumerate(geom.geoms):
            add_geom(g, f"{name}_g{i}", desc, cname, folder)

    else:
        # Fallback to point at centroid
        c = geom.centroid
        p = folder.newpoint(name=f"{name}", coords=[(c.x, c.y)])
        p.description = f"(Centroid fallback) {desc}"
        style_for_class(p, cname)

# Write features
for _, row in gdf.iterrows():
    cname = str(row["class_name"])
    fid = int(row["id"])
    name = f"{fid}: {cname}"
    desc = f"id={fid}\nclass_name={cname}\ncenter_lat={row['center_lat']:.6f}\ncenter_lon={row['center_lon']:.6f}"
    folder = ensure_folder(cname)
    add_geom(row.geometry, name, desc, cname, folder)

# Save
Path(DST).parent.mkdir(parents=True, exist_ok=True)
kml.save(DST)
print(f"KML written: {DST}")

# Optional: also persist the reduced GeoJSON alongside, if needed
reduced_geojson = str(Path(DST).with_suffix(".sustainablity_lab.geojson"))
gdf.to_file(reduced_geojson, driver="GeoJSON")
print(f"Reduced GeoJSON written: {reduced_geojson}")


KML written: punjab_selected.kml
Reduced GeoJSON written: punjab_selected.sustainablity_lab.geojson
